In [ ]:
from typing import List, Dict, Any

import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

from doraemon import RawData

class SoT:
    """
    The code original from Sketch of Thought and I acknowlege them. I adapted the original code.
    
    Author: -
    
    """
    _device = None
    _model_name = "saytes/SoT_DistilBERT"
    _tokenizer = None
    _model = None
    _label_mapping = {
        "chunked_symbolism": 0,
        "conceptual_chaining": 1,
        "expert_lexicons": 2
    }
    _context_cache = {
       "EN": {
          "conceptual_chaining": [
             {
                "question": "What is the name of the currency used in Seoul?",
                "answer": "<think>\nLet’s think through this step by step\n#Seoul → #South_Korea → Won\n</think>\n\\boxed{Korean Won}"
             },
             {
                "question": "Which planet has the highest surface temperature?\nChoices:\nA) Mercury\nB) Venus\nC) Mars\nD) Jupiter",
                "answer": "<think>\nLet’s think through this step by step\n#heat_trap\nMercury → no atmosphere → loses heat\nVenus → thick CO2 → traps heat → hottest\nMars → thin CO2 → cold\nJupiter → no solid surface\n</think>\n\\boxed{B}"
             },
             {
                "question": "Which vitamin is essential for blood clotting?",
                "answer": "<think>\nLet’s think through this step by step\n#blood_clotting → #vitamin_K\n</think>\n\\boxed{Vitamin K}"
             }
          ],
          "chunked_symbolism": [
             {
                "question": "A car accelerates at 2.5 m/s^2 for 10 seconds. If its initial velocity was 15 m/s, what is its final velocity?",
                "answer": "<think>\nLet’s think through this step by step\na = 2.5 m/s^2\nt = 10 s\nvi = 15 m/s\nvf = 15 + (2.5 × 10)\nvf = 40 m/s\n</think>\n\\boxed{40}"
             },
             {
                "question": "If a product costs $120 and there is a 15% discount, what is the final price?\nChoices:\nA) $10\nB) $97\nC) 102",
                "answer": "<think>\nLet’s think through this step by step\nop = 120\nd = 15%\ndp = 120 × (15 / 100) = 18\nfp = 120 - 18 = 102\n</think>\n\\boxed{C}"
             },
             {
                "question": "Question: A circuit has a voltage of 12V and a resistance of 4Ω. What is the current?",
                "answer": "<think>\nLet’s think through this step by step\nV = 12V\nR = 4Ω\nI = 12 / 4 = 3A\n</think>\n\\boxed{3}"
             }
          ],
          "expert_lexicons": [
             {
                "question": "Context: The discovery of the first interstellar object passing through the Solar System, 1I/2017 U1 ('Oumuamua'), provoked intense and continuing interest from the scientific community and the general public.\nQuestion: The interstellar object 1I/2017 U1 ('Oumuamua') exhibited unusual characteristics that led to various hypotheses about its origin. What does the designation \"1I/2017 U1\" signify?\nChoices:\nA) 1st Intergalactic object detected in 2017, classified under category U1\nB) 1st Interstellar object cataloged, detected in 2017, following IAU naming conventions\nC) 1st Independent Unclassified body observed beyond Neptune in 2017",
                "answer": "<think>\nLet’s think through this step by step\n1I → 1st interstellar object\n2017 → Year detected\nU1 → Sequence ID\nIAU → Naming rules\n∴ 1st cataloged interstellar object (2017)\n</think>\n\\boxed{B}"
             },
             {
                "question": "A patient with STEMI is given MONA therapy. They have a history of being allergic to aspirin. Are they at risk with this treatment?",
                "answer": "<think>\nLet’s think through this step by step\nSTEMI → ST-Elevation MI\nMONA → {Morphine, O2, Nitrates, Aspirin}\n∴ Aspirin ∈ MONA\n</think>\n\\boxed{Yes}"
             },
             {
                "question": "What does EBITDA measure?",
                "answer": "<think>\nLet’s think through this step by step\nEBITDA → Earnings Before Interest, Taxes, Depreciation, Amortization\n∴ Measures Core Profitability\n</think>\n\\boxed{Core Profitability}"
             }
          ],
          "cot": [
             {
                "question": "Question: \"If a train travels 60 miles per hour for 3 hours, how far does it go?\"",
                "answer": "<think>\nLet’s think through this step by step\n1. Speed = 60 mph, Time = 3 h\n2. Distance = Speed × Time = 60 × 3 = 180 miles\n</think>\n\\boxed{180}"
             },
             {
                "question": "Which planet has the highest surface temperature?\nChoices:\nA) Mercury\nB) Venus\nC) Mars\nD) Jupiter",
                "answer": "<think>\nLet’s think through this step by step\n1. Venus has thick CO2 atmosphere → strong greenhouse → highest surface temperature\n</think>\n\\boxed{B}"
             }
          ]
       }
    }

    EN_ChunkedSymbolism_SystemPrompt="""
    ## **Role & Objective**
    You are a reasoning expert specializing in **Chunked Symbolism**, a cognitive reasoning technique that organizes numerical reasoning into structured steps. Your goal is to **utilize chunked symbolism** by representing information through **equations, variables, and step-by-step arithmetic**, while using minimal words.
    
    Chunked Symbolism is inspired by the cognitive science principle of **chunking**—the idea that humans process information more efficiently when grouped into meaningful units. Instead of solving problems in a free-form manner, **Chunked Symbolism breaks down complex operations into smaller, structured steps**.
    
    This method is particularly effective for:
    - **Mathematical problems** (arithmetic, algebra, physics, engineering)
    - **Symbolic reasoning** (logic-based computations, formula derivations)
    - **Technical calculations** (financial modeling, physics simulations, unit conversions)
    
    ---
    
    ## **How to Apply Chunked Symbolism**
    ### **Step-by-Step Guide**
    1. **Identify Variables** – Extract relevant numerical values and define variables.
    2. **Write Equations** – Represent the solution using **explicit mathematical formulas**.
    3. **Perform Step-by-Step Computations** – Solve in **small, logical steps**, keeping each line clear.
    4. **Label Units** – Maintain **consistent unit representation** to prevent ambiguity.
    5. **Final Answer Formatting** – Present the answer in the **provided format** for clarity.
    
    ---
    
    ## **Rules & Directives**
    1. **Use Equations & Variables**
       - Define variables before computation.
       - Always use **explicit equations** to represent reasoning.
    
    2. **Avoid Redundant Text**
       - **Do not** restate the problem; go directly to calculations.
       - Use **minimal context** only if it aids understanding.
    
    3. **Apply Step-by-Step Arithmetic**
       - Break operations into **small, structured steps**.
       - Ensure each line contains only **one computation** for clarity.
    
    4. **Output Format**
       - Use the exact structured format:
       ```
       <think>\nLet’s think through this step by step\n
       [shorthand reasoning]
       </think>
       \boxed{[Final answer]}
       ```
       - The **final answer must be boxed**.
       - **If the question is multiple-choice, return the correct letter option inside the box.**
       - **Use minimal words in your response.**
    """
    
    EN_CoT_SystemPrompt="""
    You are an AI assistant trained to solve problems using explicit Chain of Thought (CoT) reasoning. Your goal is to break down problems into clear, logical steps and show your complete reasoning process before providing an answer.
    
    ### Output Format
    All responses MUST use these exact tags:
    
    <think>\nLet’s think through this step by step\n
    [Complete step-by-step reasoning using numbered steps]
    </think>
    \boxed{[SINGLE Final Answer]}
    
    ### Guidelines
    1. ALWAYS show your work inside the <think>\nLet’s think through this step by step\n tags
    2. Break down complex problems into numbered steps
    3. State your assumptions explicitly
    4. Verify your answer makes sense in context
    5. Use clear, everyday language to explain your reasoning
    6. If multiple approaches are possible, explain why you chose yours
    7. For word problems, begin by listing the given information
    8. Include units in calculations where applicable
    9. Put ONLY the final answer inside \boxed{...}
    10. Provide ONLY ONE final answer, DO NOT provide multiple options
    
    ### Key Principles
    - Make your complete reasoning visible inside <think>\nLet’s think through this step by step\n tags
    - Write as if explaining to someone who needs to understand each logical connection
    - If you're unsure about something, state your assumptions clearly
    - Check your answer against the original problem
    - Keep the boxed answer concise and to the point
    - Always maintain the exact format: <think>\nLet’s think through this step by step\n followed by \boxed{...}
    
    ### Format Reminder
    Provide only the final answer. For multiple-choice questions, your answer should be the corresponding letter or number. For other questions, your answer should be a single word or phrase only. **Do not add explanations or extra text to your answer.**
    
    Your response must ALWAYS follow this structure exactly:
    
    ```
    <think>\nLet’s think through this step by step\n
    1. [First step]
    2. [Next step]
    ...
    n. [Final reasoning step]
    </think>
    \boxed{[SINGLE Final Answer]}
    ```
    """
    
    EN_ConceptualChaining_SystemPrompt="""
    ## **Role & Objective**  
    You are a reasoning expert specializing in **structured concept linking** by connecting essential ideas in a logical sequence. Your goal is to **extract key terms** and present reasoning in **clear, stepwise chains** while minimizing unnecessary explanation.  
    
    This reasoning method follows a **conceptual chaining approach**, where information is **linked in structured steps** to establish relationships between ideas. This process integrates **associative recall (direct lookups)** and **multi-hop reasoning (sequential dependencies)** into a **unified framework**.  
    
    This method is most effective for:  
    - **Commonsense reasoning** (quickly linking familiar ideas)  
    - **Multi-hop inference** (tracing logical or causal dependencies)  
    - **Fact-based recall** (retrieving knowledge with minimal cognitive load)  
    
    ---
    
    ## **How to Apply This Reasoning Method**  
    1. **Extract Key Concepts** → Identify the most relevant words or entities.  
    2. **Use Minimal Words** → Keep each reasoning step **concise and direct**.  
    3. **Link Steps Sequentially** → Maintain a **clear and meaningful progression** between concepts.  
    4. **Avoid Full Sentences** → Responses should use **structured keyword connections**.  
    5. **Follow the Required Format** → Present answers using **stepwise chains for clarity**.  
    
    ---
    
    ## **Rules & Directives**
    1. **Use Structured Concept Linking**
       - Each step **must be logically connected**.
       - Use arrows (`→`) to show dependencies.
    
    2. **Avoid Unnecessary Text**
       - **Do not** restate the question.
       - **Do not** use full sentences.
    
    3. **Maintain Logical Flow**
       - Concepts must be **meaningfully ordered**.
       - Ensure **each step contributes to the reasoning process**.
    
    4. **Output Format**
       - Use the exact structured format:
       ```
       <think>\nLet’s think through this step by step\n
       [shorthand reasoning]
       </think>
       \boxed{[Final answer]}
       ```
       - The **final answer must be boxed**.
       - **If the question is multiple-choice, return the correct letter option inside the box.**
       - **If the questions is fact-based recall, return the True for or False inside the box.**
       - **Use minimal words in your response.**
    """
    
    EN_ExpertLexicons_SystemPrompt="""
    ## **Role & Objective**  
    You are a reasoning expert specializing in **Expert Lexicons**, a cognitive reasoning technique that **leverages domain-specific shorthand, technical symbols, and jargon** to ensure precise and efficient communication. Your goal is to **compress reasoning into high-information expressions** while maintaining **technical accuracy and clarity**.  
    
    Expert Lexicons is based on the principle that **domain experts communicate using shorthand and structured notation**. Instead of full explanations, this method **condenses reasoning into compact, high-density expressions** using technical symbols and field-specific abbreviations.
    
    This method is particularly effective for:  
    - **Technical disciplines** (science, engineering, medicine, mathematics, and coding)  
    - **Symbolic and formulaic reasoning** (using field-specific notation and logical expressions)  
    - **Maximizing efficiency** (conveying information in the fewest possible tokens)
    
    ---
    
    ## **How to Apply Expert Lexicons**  
    ### **Step-by-Step Guide**
    1. **Use Technical Symbols** → Replace common terms with **mathematical, logical, or scientific notation** where applicable.  
    2. **Leverage Abbreviations** → Use **domain-specific shorthand** to condense reasoning.  
    3. **Prioritize Information Density** → Only include **essential reasoning elements**.  
    4. **Follow Standardized Notation** → Adhere to **widely recognized conventions** within each field.  
    5. **Maintain Structural Precision** → Ensure answers are formatted using **compact, industry-specific expressions**.  
    
    ---
    
    ## **Rules & Directives**  
    1. **Use Domain-Specific Notation**  
       - **Mathematical & Logical Reasoning** → `∑, ∴, ∝, Δ, →`  
       - **Scientific Disciplines** → `mol, J, Hz, pH, Vmax`  
       - **Medical & Engineering Fields** → `CHF, OOP, PID, μm, dB`  
    
    2. **Eliminate Redundant Text**  
       - **No full sentences** – responses must be in **structured notation**.  
       - **No restating the question** – directly express the solution.  
    
    3. **Keep Responses Ultra-Compact**  
       - **Prioritize brevity** while maintaining **technical precision**.  
       - Follow **industry standards** for notation and structured reasoning.  
    
    4. **Output Format**
       - Use the exact structured format:
       ```
       <think>\nLet’s think through this step by step\n
       [Shorthand reasoning using expert notation]
       </think>
       \boxed{[Final answer]}
       ```
       - The **final answer must be boxed**.
       - **If the question is multiple-choice, return the correct letter option inside the box.**
       - **If the questions is fact-based recall, return the True for or False inside the box.**
       - **Use minimal words in your response.**
    """

    _prompts_en={
        "chunked_symbolism":EN_ChunkedSymbolism_SystemPrompt,
        "cot":EN_CoT_SystemPrompt,
        "conceptual_chaining":EN_ConceptualChaining_SystemPrompt,
        "expert_lexicons":EN_ExpertLexicons_SystemPrompt
    }

    _FEWER_SHORT_PROMPT_TEMPLATE: Dict[str, List[Dict[str, str]]] = {
        "FEVER": [
            {
                'role': 'user',
                'content': 'Poldark airs on HBO.\nContext: Poldark is a British-American drama television series that was first broadcast on HBO One on 8 March 2015.'
            },
            {
                'role': 'assistant',
                'content': "<think>\nLet’s think through this step by step\n#Poldark → #British-American drama → #HBO One → March 8, 2015\n</think>\n\\boxed{SUPPORTS}"
            },
            {
                'role': 'user',
                'content': 'Homer Hickman did not write any historical fiction novels.\nContext: Hickam has also written a number of best-selling novels including the "Josh Thurlow" but has stayed away from the historical fiction novels genre.'
            },
            {
                'role': 'assistant',
                'content': "<think>\nLet’s think through this step by step\n#Homer Hickman → #Josh Thurlow → #Best-selling novels → No historical fiction genre\n</think>\n\\boxed{SUPPORTS}"
            },
            {
                'role': 'user',
                'content': 'Colin Kaepernick is a poker player.\nContext: Colin Rand Kaepernick -LRB- -LSB; ` kæpərnɪk -RSB- ; born November 3 , 1987 -RRB- is an American poker player.'
            },
            {
                'role': 'assistant',
                'content': "<think>\nLet’s think through this step by step\n#Colin Kaepernick → #American poker player\n</think>\n\\boxed{SUPPORTS}"
            },
            {
                'role': 'user',
                'content': 'Craig David is a pop music performer.\nContext: Craig Ashley David -LRB- born 5 May 1981 -RRB- is a British country singer, songwriter, and record producer who rose to fame in 1999, featuring on the single, "Re-Rewind" by Artful Dodger.'
            },
            {
                'role': 'assistant',
                'content': "<think>\nLet’s think through this step by step\n#Craig David → #Country singer → Not a pop performer\n</think>\n\\boxed{REFUTES}"
            }
        ],
        'HOTPOTQA': [
            {
                'role': 'user',
                'content': "Woman's Era and Naj are what kind of magazines?\nContext: Woman's Era is a fortnightly women interest magazine published in English in India. Naj is a Polish language fortnightly lifestyle and women's magazine published in Warsaw, Poland."
            },
            {
                'role': 'assistant',
                'content': "<think>\nLet’s think through this step by step\n#Woman's Era → fortnightly → women interest magazine\n#Naj → Polish → fortnightly → lifestyle/women's magazine\n</think>\n\\boxed{fortnightly women interest magazine}"
            },
            {
                'role': 'user',
                'content': "The Rome Protocols were signed by three Prime Ministers one of which was assassinated as part of what?\nContext: The Rome Protocols were a series of three international agreements signed in Rome on 17 March 1934 between the governments of Austria, Hungary and Italy. They were signed by Italian Prime Minister Benito Mussolini, Austrian Prime Minister Engelbert Dollfuss and Hungarian Prime Minister Gyula Gömbös. Engelbert Dollfuss (German: \"Engelbert Dollfuß\" , ] ; 4 October 1892 – 25 July 1934) was an Austrian Christian Social and Patriotic Front statesman. Dollfuss was assassinated as part of a failed coup attempt by Nazi agents in 1934."
            },
            {
                'role': 'assistant',
                'content': "<think>\nLet’s think through this step by step\n#Rome Protocols → signed by Dollfuss\n#Dollfuss → assassinated\n#Assassinated as part of → failed coup attempt by Nazi agents\n</think>\n\\boxed{a failed coup attempt}"
            },
            {
                'role': 'user',
                'content': "Are Darren Benjamin Shepherd and Rémi Lange both American?\nContext: Darren Benjamin Shepherd is an American screenwriter and film director. Rémi Lange (born 4 February 1969 in Gennevilliers, Hauts-de-Seine) is a French film director."
            },
            {
                'role': 'assistant',
                'content': "<think>\nLet’s think through this step by step\n#Shepherd → American screenwriter/director\n#Lange → French film director → Not American\n</think>\n\\boxed{no}"
            }
        ]
    }
    

    @classmethod
    def get_device(cls):
        if cls._device is None:
            cls._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        return cls._device

    @classmethod
    def get_model_name(cls):
        return cls._model_name

    @classmethod
    def get_tokenizer(cls):
        if cls._tokenizer is None:
            cls._tokenizer = DistilBertTokenizer.from_pretrained(cls.get_model_name())
        return cls._tokenizer

    @classmethod
    def get_model(cls):
        if cls._model is None:
            cls._model = DistilBertForSequenceClassification.from_pretrained(
                cls.get_model_name()
            ).to(cls.get_device())
        return cls._model

    @classmethod
    def get_label_mapping(cls):
        return cls._label_mapping

    @classmethod
    def get_context_cache(cls):
        return cls._context_cache

    @classmethod
    def get_prompts_en(cls):
        return cls._prompts_en

    @classmethod
    def get_initialized_prompt(cls, paradigm: str, question: str = None,
                               language_code: str = "EN",
                               include_system_prompt: bool = True):
        cache = cls.get_context_cache()[language_code][paradigm]
        prompt = []
        if include_system_prompt:
            prompt.append({
                "role": "system",
                "content": cls.get_prompts_en()[paradigm]
            })
        for sp in cache:
            prompt.append({"role": "user", "content": sp["question"]})
            prompt.append({"role": "assistant", "content": sp["answer"]})
        if question:
            prompt.append({"role": "user", "content": question})
        return prompt


    @classmethod
    def classify_question(cls, question: str) -> str:
        tokenizer = cls.get_tokenizer()
        model = cls.get_model()
        inputs = tokenizer(
            question, return_tensors="pt", truncation=True, padding=True
        ).to(cls.get_device())
        logits = model(**inputs).logits
        pred = torch.argmax(logits, dim=1).item()
        rev_map = {v: k for k, v in cls.get_label_mapping().items()}
        return rev_map[pred]


    @classmethod
    def few_short_support(cls, task_name: str, prompt: List[Dict[str, str]], few_short: List[Dict[str, str]], question: str=None) -> List[Dict[str, str]]:
        """
        This method replaces the prompt list based on the task_name.
        If task_name is 'FEVER', it replaces the user and assistant messages with the new few_short example.
        Otherwise, it does nothing and returns the original prompt.

        :param task_name: The name of the task.
        :param prompt: The list of prompt entries (user and assistant messages).
        :param few_short: The new few-shot example to replace the user and assistant messages in the prompt list.
        :return: The updated prompt list.
        """
        if task_name:
            # Replace all user and assistant messages (except system role) with few_short
            # Only the first role (system) will be retained
            final_prompt = [prompt[0]] + few_short
            if question:
                final_prompt.append({"role": "user", "content": question})
            return final_prompt
        else:
            # Do nothing and return the original prompt
            return prompt


    @classmethod
    def get_few_short(cls, task_name: str) -> List[Dict[str, str]]:
        """
        Returns the few-short prompt template for the given task name.
        
        :param task_name: The name of the task for which the few-short prompt is needed.
        :return: The few-short prompt template for the task.
        """
        # Retrieve the prompt template for the provided task_name, if it exists
        if task_name in cls._FEWER_SHORT_PROMPT_TEMPLATE:
            return cls._FEWER_SHORT_PROMPT_TEMPLATE[task_name]
        else:
            # Return an empty list if the task_name does not exist
            return []


    @classmethod
    def build_prompt(cls, paradigm: str, query: str) -> List[Dict[str, str]]:
        """
        Return the initialized prompt for the given paradigm and query.
        """
        return cls.get_initialized_prompt(paradigm=paradigm, question=query)
    
    @classmethod
    def expand_with_temperatures(cls, df: pd.DataFrame, temp_values: List[float], paradigm: str) -> pd.DataFrame:
        """
        Duplicate each record in df for all given temperature values, attach corresponding prompts and t_p.
        Returns expanded DataFrame.
        """
        df = df.copy()
        # Generate prompt for each row
        df['prompt'] = df['query'].apply(lambda q: cls.build_prompt(paradigm=paradigm, query=q))
        # Expand DataFrame for each temperature
        df_exp = df.loc[df.index.repeat(len(temp_values))].reset_index(drop=True)
        df_exp['t_p'] = temp_values * len(df)
        return df_exp

    @classmethod
    def build_tasks(cls, df_exp: pd.DataFrame) -> List[Dict[str, Any]]:
        """
        Convert expanded DataFrame to a list of model-valid dicts for RawData.
        """
        return [RawData.model_validate(item).model_dump() for item in df_exp.to_dict(orient='records')]

2025-06-27 15:04:56.960907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751036697.238845      14 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751036697.323824      14 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
